In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms


3.2 MNIST数据集手写数字识别
3.2.1 数据集介绍
MNIST 包括6万张28x28的训练样本，1万张测试样本，很多教程都会对它”下手”几乎成为一个 “典范”，可以说它就是计算机视觉里面的Hello World。所以我们这里也会使用MNIST来进行实战。

前面在介绍卷积神经网络的时候说到过LeNet-5，LeNet-5之所以强大就是因为在当时的环境下将MNIST数据的识别率提高到了99%，这里我们也自己从头搭建一个卷积神经网络，也达到99%的准确率

3.2.2 手写数字识别
首先，我们定义一些超参数

In [16]:
BATCH_SIZE = 512 #大概需要2G显存
EPOCHS = 20#迭代20次
DEVICE = torch.device('cuda'if torch.cuda.is_available() else 'cpu') #让torch判断是否使用GPU

因为Pytorch里面包含了MNIST的数据集，所以我们这里直接使用即可。 如果第一次执行会生成data文件夹，并且需要一些时间下载，如果以前下载过就不会再次下载了

由于官方已经实现了dataset，所以这里可以直接使用DataLoader来对数据进行读取

In [17]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data',train=True,download=True,
                   transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,),(0.3081,))
                   ])),
    batch_size = BATCH_SIZE,shuffle = True)

In [18]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
    batch_size=BATCH_SIZE, shuffle=True)

下面我们定义一个网络，网络包含两个卷积层，conv1和conv2，然后紧接着两个线性层作为输出，最后输出10个维度，这10个维度我们作为0-9的标识来确定识别出的是那个数字

在这里建议大家将每一层的输入和输出维度都作为注释标注出来，这样后面阅读代码的会方便很多

In [19]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        #batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        #下面的卷积层Conv2d的第一个参数指定输入通道数，第二个参数指定输出通道数，第三个参数指定卷积核的大小
        self.conv1 = nn.Conv2d(1,10,5)# ((n-f+2p)/s)+1 向下取整
        self.conv2 = nn.Conv2d(10,20,3)
        #下面的全连接层第一个参数指定输入特征数，第二个参数指定输出特征数
        self.fc1 = nn.Linear(20*10*10,500)
        self.fc2 = nn.Linear(500,10)
    def forward(self,x):
        in_size = x.size(0)#在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量
        out = self.conv1(x)# batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
        out = F.relu(out) # batch*10*24*24（激活函数ReLU不改变形状））
        out = F.max_pool2d(out,2,2)# batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
        out = self.conv2(out)# batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
        out = F.relu(out)# batch*20*10*10
        out = out.view(in_size,-1)# batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10）
        out = self.fc1(out) # batch*2000 -> batch*500
        out = F.relu(out)# batch*500
        out = self.fc2(out)# batch*500 -> batch*10
        out = F.log_softmax(out,dim=1)# 计算log(softmax(x)) dim=1表示按列计算
        return out

我们实例化一个网络，实例化后使用.to方法将网络移动到GPU

优化器我们也直接选择简单暴力的Adam



In [20]:
model = ConvNet().to(DEVICE)
optim = optim.Adam(model.parameters())

In [21]:
def train(model,device,train_loader,optim,epoch):
    model.train()#指定训练模式
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target = data.to(device),target.to(device)
        optim.zero_grad()
        output = model(data)
        loss = F.nll_loss(output,target)
        loss.backward()
        optim.step()
        if (batch_idx+1)%30 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [22]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [23]:
for epoch in range(1,EPOCHS+1):
    train(model,DEVICE,train_loader,optim,epoch)
    test(model,DEVICE,test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.316750
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.209471
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.161886

Test set: Average loss: 0.0899, Accuracy: 9734/10000 (97%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.078537
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.066543
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.079835

Test set: Average loss: 0.0660, Accuracy: 9782/10000 (98%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.031043
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.042085
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.087403

Test set: Average loss: 0.0412, Accuracy: 9867/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.044869
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.074912
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.049452

Test set: Average loss: 0.0453, Accuracy: 9845/10000 (98%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.026344
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.036668
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.028507

Test 